In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
sns.set_style("darkgrid")

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
sub = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")

In [ ]:
display(train.head()),
display(test.head()),
display(sub.head())

In [ ]:
testoriginal = test.copy()

In [ ]:
print(train.shape,test.shape,sub.shape)

In [ ]:
train.isnull().sum()

In [ ]:
train = train.drop(['id'] , axis = 1)
test = test.drop(['id'] , axis = 1)

In [ ]:
display(train.describe())
display(test.describe())

In [ ]:
train['Encoded Target'] = train['target'].map({'Class_1':0, 'Class_2':1, 'Class_3':2, 'Class_4':3})


In [ ]:
a = train.drop(['target','Encoded Target'], axis=1)
a.corrwith(train['Encoded Target']).plot(kind='bar', figsize=(18,11), color=['blue'])
plt.xticks(size=15)
plt.yticks(size=15)
plt.show()               


In [ ]:
X = train.iloc[:,:-2]

Y = train['Encoded Target']

In [ ]:
x_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.15,random_state = 48)

In [ ]:
xgbclassifier = XGBClassifier(learning_rate =0.2,
              n_estimators =100,
              max_depth=8,
              min_child_weight=3,
              gamma=0.3,
              subsample=0.85,
              colsample_bytree=0.8,
              objective= 'multi:softprob',
              nthread=4,
              eval_metric = 'mlogloss',
              random_state=42 ,use_label_encoder=True)


In [ ]:
xgbclassifier = xgbclassifier.fit(X, Y)

In [ ]:
y_pred_xgb = xgbclassifier.predict_proba(X_test)
y_pred_xgb_test = xgbclassifier.predict_proba(test)

print(log_loss(y_test, y_pred_xgb))

In [ ]:
plt.rcParams["figure.figsize"] = (15, 15)
xgb.plot_importance(xgbclassifier)
plt.show()

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(xgbclassifier, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist(),top = 50)

In [ ]:
import shap 
explainer = shap.TreeExplainer(xgbclassifier)


shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values[1], X_test)

In [ ]:
shap_values_test = explainer.shap_values(test)

shap.summary_plot(shap_values_test[1], test)

In [ ]:
shap.dependence_plot('feature_38', shap_values_test[1], test, interaction_index="feature_15")

In [ ]:
submission = pd.DataFrame(y_pred_xgb_test)
submission.columns = ["Class_1", "Class_2", "Class_3","Class_4"]
submission.insert(0, 'id', testoriginal['id'])
submission.to_csv("submission.csv", index = False)